# import libraries

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import itertools
import csv,codecs,nltk,re
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import itertools
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, NuSVC , LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier

# Read the dataset

In [2]:
# -----  New Dataset ------------
data = pd.read_csv(r"D:\Cyber_2021-1.csv" ,sep=";",error_bad_lines=False)
data.columns = ["text","Label"]
#data.dropna(inplace=True)

b'Skipping line 241: expected 2 fields, saw 3\n'


In [2]:
# -----  Hamdy Dataset ------------
data = pd.read_csv(r"D:\Dataset\Test\Unbalanced\Arabic\Hamdy\Hamdy-Twitter.csv" ,sep=";",encoding="utf-8")
data.columns = ["text","Label"]

In [24]:
# -----  Al akroot Dataset ------------
data = pd.read_csv(r"D:\Dataset\Test\Unbalanced\Arabic\Alakrot\Alakrot-Youtube.csv" ,sep=";",encoding="utf-8")
data.columns = ["text","Label"]

In [3]:
len(data)

13249

In [3]:
data.head()

,text,Label
0,مبروك و سامحونا لعجزنا التام. عقبال اللي جوه. ...,0
1,كلنا بره ومش هنبطل نزايد على العجايز الي جابون...,1
2,بدل ما انت قاعد بره كده تعالي ازرع الصحرا,0
3,قذر اتفووو ماتيجى مصر وتورينا نفسك كدا ياجبان,1
4,وهكذا رجال الشو اللي محرومين من عمل برنامج الغ...,1


In [3]:
data.Label.value_counts()

 1    6859
0     6355
0B       1
Name: Label, dtype: int64

In [3]:
data=data[data.Label !='0B']

# text preprocessing (Arabic)

In [4]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("_", " ", text)
    text = re.sub("#", " ", text)
    
    text = re.sub("ؤ", "و", text)
    #text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    return(text)
def stopwordremoval(text):
    stop=stopwords.words("arabic")
    needed_words=[]
    words=word_tokenize(text)
    for w in words:
         if len(w)>=2 and w not in stop:
                needed_words.append(w)
    filterd_sent= " ".join(needed_words)
    return filterd_sent
def removenonarabic(text):
    n=re.sub(r'[a-zA-Z?]', '', text).strip()
    n=re.sub('\W+',' ', n )
    n=re.sub('_','', n )
    n = ''.join([i for i in n if not i.isdigit()])
    return n
def stemming(text):
    st = ISRIStemmer()
    stemmed_words=[]
    words=word_tokenize(text)
    for w in words:
        stemmed_words.append(st.stem(w))
    stemmed_sent=" ".join(stemmed_words)
    return stemmed_sent
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
def preparedatasets(data):
    sentences=[]
    for index,r in data.iterrows():
        text=normalize(r['text'])
        text=stopwordremoval(text)
        text=removenonarabic(text)
        #text=stemming(text)
        sentences.append([text,r['Label']])
    df_sentence=DataFrame(sentences, columns=["text", "Label"])
    return df_sentence        

In [5]:
from nltk.corpus import stopwords
stopwords.words('arabic')

['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

In [6]:
data=preparedatasets(data)


In [7]:
data=preparedatasets(data)
data.dropna(inplace=True)
data.head()
data['word_count'] = data['text'].apply(lambda x: len(str(x).split()))
#Remove 0 and 1 word_count posts
new_data=data[(data.word_count >1)]
new_data.describe()    
data=new_data


In [8]:
data=data.drop_duplicates( keep="first")

In [9]:
data.Label.value_counts()

1    637
0    450
Name: Label, dtype: int64

In [10]:
#Number of words in the dataset
def get_count(data):
    d2=[]
    tt=""
    for index,r in data.iterrows():
        d2.append((r['text'] ))
        tt+=r['text']
    return len(tt)    

In [11]:
X_train, X_test, y_train, y_test = \
train_test_split(data['text'], data['Label'],test_size=0.20, random_state = 0)
vectorizer = TfidfVectorizer( analyzer='word',smooth_idf=True, ngram_range=(1,2))
vectorizer.fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)   

In [14]:
# ---------- Voting ---- 
from sklearn.ensemble import VotingClassifier
model1 = KNeighborsClassifier()
model2= DecisionTreeClassifier()
model3=LogisticRegression()
model4=LinearSVC() 

model1.fit(X_train_vectorized,y_train)
model2.fit(X_train_vectorized,y_train)
model3.fit(X_train_vectorized,y_train)
model4.fit(X_train_vectorized,y_train)

model = VotingClassifier(estimators=[('dt', model1),('rf', model2),('ml', model3),('lsv', model4)], voting='hard')
model.fit(X_train_vectorized,y_train)
model.score(vectorizer.transform(X_test),y_test)

predictions = model.predict(vectorizer.transform(X_test))
print("accuracy",accuracy_score(y_test, predictions))
print("f1_score",f1_score(y_test, predictions,average='weighted'))
print("recall",recall_score(y_test, predictions,average='weighted'))
print("precision",precision_score(y_test, predictions,average='weighted'))    
print ('\n clasification report:\n', classification_report(y_test, predictions))
print (' confussion matrix:\n',confusion_matrix(y_test, predictions))


C:\Users\NoteBook\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy 0.7018348623853211
f1_score 0.7014814518453775
recall 0.7018348623853211
precision 0.7011578126439656

 clasification report:
               precision    recall  f1-score   support

           0       0.61      0.60      0.61        83
           1       0.76      0.76      0.76       135

    accuracy                           0.70       218
   macro avg       0.68      0.68      0.68       218
weighted avg       0.70      0.70      0.70       218

 confussion matrix:
 [[ 50  33]
 [ 32 103]]


C:\Users\NoteBook\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [15]:
# --- ---------- Bagging ----------------
# 1- Desicion Tree
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(X_train_vectorized, y_train)
model.score(vectorizer.transform(X_test),y_test)

predictions = model.predict(vectorizer.transform(X_test))
print("accuracy",accuracy_score(y_test, predictions))
print("f1_score",f1_score(y_test, predictions,average='weighted'))
print("recall",recall_score(y_test, predictions,average='weighted'))
print("precision",precision_score(y_test, predictions,average='weighted'))    
print ('\n clasification report:\n', classification_report(y_test, predictions))
print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

accuracy 0.9601873536299765
f1_score 0.9601869896976261
recall 0.9601873536299765
precision 0.9601971586286903

 clasification report:
               precision    recall  f1-score   support

           1       0.96      0.96      0.96      1283
           0       0.96      0.96      0.96      1279

    accuracy                           0.96      2562
   macro avg       0.96      0.96      0.96      2562
weighted avg       0.96      0.96      0.96      2562

 confussion matrix:
 [[1235   48]
 [  54 1225]]


In [16]:
 # 2- ------  Random- forest -----
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier()

# fit the model with the training data
model.fit(X_train_vectorized, y_train)

# number of trees used
print('Number of Trees used : ', 100)

# predict the target on the train dataset
predict_train = model.predict(X_train_vectorized)
#print('\nTarget on train data',predict_train) 

# Accuray Score on train dataset
accuracy_train = accuracy_score(y_train,predict_train)
print('\naccuracy_score on train dataset : ', accuracy_train)

# predict the target on the test dataset
predict_test = model.predict(vectorizer.transform(X_test))
#print('\nTarget on test data',predict_test) 

# Accuracy Score on test dataset
accuracy_test = accuracy_score(y_test,predict_test)
print('\naccuracy_score on test dataset : ', accuracy_test)


print("accuracy",accuracy_score(y_test, predict_test))
print("f1_score",f1_score(y_test, predict_test,average='weighted'))
print("recall",recall_score(y_test, predict_test,average='weighted'))
print("precision",precision_score(y_test, predict_test,average='weighted'))    
print ('\n clasification report:\n', classification_report(y_test, predict_test))
print (' confussion matrix:\n',confusion_matrix(y_test, predict_test))

C:\Users\NoteBook\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Number of Trees used :  100

accuracy_score on train dataset :  0.9988290398126464

accuracy_score on test dataset :  0.9613583138173302
accuracy 0.9613583138173302
f1_score 0.9613580488967537
recall 0.9613583138173302
precision 0.9613651073205672

 clasification report:
               precision    recall  f1-score   support

           1       0.96      0.96      0.96      1283
           0       0.96      0.96      0.96      1279

    accuracy                           0.96      2562
   macro avg       0.96      0.96      0.96      2562
weighted avg       0.96      0.96      0.96      2562

 confussion matrix:
 [[1236   47]
 [  52 1227]]


In [17]:
# ----------  Booosting -------
#----Boosting -- AdaBoost 
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(X_train_vectorized, y_train)
model.score(vectorizer.transform(X_test),y_test)

predictions = model.predict(vectorizer.transform(X_test))
print("accuracy",accuracy_score(y_test, predictions))
print("f1_score",f1_score(y_test, predictions,average='weighted'))
print("recall",recall_score(y_test, predictions,average='weighted'))
print("precision",precision_score(y_test, predictions,average='weighted'))    
print ('\n clasification report:\n', classification_report(y_test, predictions))
print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

accuracy 0.9480874316939891
f1_score 0.9480617555879797
recall 0.9480874316939891
precision 0.9490435937254006

 clasification report:
               precision    recall  f1-score   support

           1       0.97      0.93      0.95      1283
           0       0.93      0.97      0.95      1279

    accuracy                           0.95      2562
   macro avg       0.95      0.95      0.95      2562
weighted avg       0.95      0.95      0.95      2562

 confussion matrix:
 [[1187   96]
 [  37 1242]]


In [18]:
#----Boosting -- GradientBoost
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(X_train_vectorized, y_train)
model.score(vectorizer.transform(X_test),y_test)

predictions = model.predict(vectorizer.transform(X_test))
print("accuracy",accuracy_score(y_test, predictions))
print("f1_score",f1_score(y_test, predictions,average='weighted'))
print("recall",recall_score(y_test, predictions,average='weighted'))
print("precision",precision_score(y_test, predictions,average='weighted'))    
print ('\n clasification report:\n', classification_report(y_test, predictions))
print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

accuracy 0.7997658079625293
f1_score 0.792640221338046
recall 0.7997658079625293
precision 0.848091038112197

 clasification report:
               precision    recall  f1-score   support

           1       0.98      0.61      0.75      1283
           0       0.72      0.99      0.83      1279

    accuracy                           0.80      2562
   macro avg       0.85      0.80      0.79      2562
weighted avg       0.85      0.80      0.79      2562

 confussion matrix:
 [[ 788  495]
 [  18 1261]]


In [19]:
#----Boosting -- XGBoost
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(X_train_vectorized, y_train)
model.score(vectorizer.transform(X_test),y_test)

predictions = model.predict(vectorizer.transform(X_test))
print("accuracy",accuracy_score(y_test, predictions))
print("f1_score",f1_score(y_test, predictions,average='weighted'))
print("recall",recall_score(y_test, predictions,average='weighted'))
print("precision",precision_score(y_test, predictions,average='weighted'))    
print ('\n clasification report:\n', classification_report(y_test, predictions))
print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

C:\Users\NoteBook\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:39:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.8645589383294301
f1_score 0.8626891565103247
recall 0.8645589383294301
precision 0.8858993569343891

 clasification report:
               precision    recall  f1-score   support

           1       0.98      0.75      0.85      1283
           0       0.79      0.98      0.88      1279

    accuracy                           0.86      2562
   macro avg       0.89      0.86      0.86      2562
weighted avg       0.89      0.86      0.86      2562

 confussion matrix:
 [[ 959  324]
 [  23 1256]]
